In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import numpy as np
import os

In [5]:
# Unzip data

In [3]:
zip_path = "Dataset.zip"
extract_path = "Dataset"
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)


In [7]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 16
EPOCHS = 3
DATASET_DIR = "Dataset"


In [11]:
# Data Generators
# ====================
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = train_datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 8404 images belonging to 1 classes.
Found 2100 images belonging to 1 classes.


In [13]:
# Model Builders
# =========================
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

def build_mobilenet_model(input_shape, num_classes):
    base = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')
    base.trainable = False
    x = Dense(128, activation='relu')(base.output)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=base.input, outputs=output)
    return model

In [15]:
# Train & Evaluate
models_to_train = [
    ("CNN_Scratch", build_cnn_model((*IMG_SIZE, 3), train_gen.num_classes)),
    ("MobileNet", build_mobilenet_model((*IMG_SIZE, 3), train_gen.num_classes))
]

best_acc = 0
best_model = None

for name, model in models_to_train:
    print(f"\nTraining {name}...")
    model.compile(optimizer=Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"])
    history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, verbose=1)
    
    val_acc = history.history['val_accuracy'][-1]
    if val_acc > best_acc:
        best_acc = val_acc
        best_model = model

C:\Users\HP\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 10s 1us/step

Training CNN_Scratch...


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3


C:\Users\HP\anaconda3\Lib\site-packages\keras\src\ops\nn.py:938: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\Users\HP\anaconda3\Lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


526/526 ━━━━━━━━━━━━━━━━━━━━ 2106s 4s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/3
526/526 ━━━━━━━━━━━━━━━━━━━━ 1975s 4s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/3
526/526 ━━━━━━━━━━━━━━━━━━━━ 1939s 4s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00

Training MobileNet...
Epoch 1/3
526/526 ━━━━━━━━━━━━━━━━━━━━ 1120s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/3
526/526 ━━━━━━━━━━━━━━━━━━━━ 957s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/3
526/526 ━━━━━━━━━━━━━━━━━━━━ 921s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [17]:
# Save Best Model
best_model.save("best_fish_model.h5")
print(f"Best model saved with accuracy: {best_acc:.4f}")

Best model saved with accuracy: 1.0000


In [21]:
import streamlit as st
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np
import os

In [23]:
# CONFIG

IMG_SIZE = (128, 128)
MODEL_PATH = "best_fish_model.h5" 
CLASS_NAMES = ["Catfish", "Salmon", "Tuna", "Trout", "Carp"]


In [25]:
# LOAD MODEL

@st.cache_resource
def load_model():
    return tf.keras.models.load_model(MODEL_PATH)

model = load_model()

In [48]:
# STREAMLIT UI
st.title("Fish Classifier")
st.write("Upload a fish image and I will tell you the type with confidence score.")

uploaded_image = st.file_uploader("Upload a fish image", type=["jpg", "jpeg", "png"])

In [50]:
if uploaded_image:
    # Display uploaded image
    st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)

    # Preprocess image 
    img = load_img(uploaded_image, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)